# Workflow #

The pipeline consists of (i), creation or reloading of the work area via Get_Data(), (ii) creation of the repertoire classification model via Monte_Carlo_CrossVal(), and (iii) representation of the AUC-ROC curve. In case of positive results in the classification model, the pipeline has continued with (iv) identification of motifs by Motif_Identification() and obtaining the Residue Sensitivity Logos for each cluster (v). This workflow has been applied in all supervised analysis scripts. The procedure has been developed given the scripts deposited at https://github.com/sidhomj/DeepTCR_COVID19, whose results are published at https://doi.org/10.1038/s41598-021-93608-8

**Important** The supervised model analyses stored in the .ipynb *cluster1vcluster2*, *cluster1vcluster3*, *cluster2vcluster3* and *severity* scripts have the same structure and code blocks (except the RSL logo code blocks in *cluster2vcluster3* and *severity* files), with corresponding differences in terms of sample directories, variable and file names. Therefore, for further simplification, more detailed workflow comments have been added in the *cluster1vcluster3.ipynb* script, applicable to the rest of the scripts, which retain basic documentation.

In [ ]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../../')
from DeepTCR.DeepTCR import DeepTCR_WF
import pickle

In [ ]:
DTCR_CT12 = DeepTCR_WF('CT1_CT2')
classes = ['ct1', 'ct2'] # Definition of classes of samples we compare
DTCR_CT12.Get_Data('ct1_ct2',
              Load_Prev_Data=False,
              aa_column_beta=0,
              v_beta_column=2,
              j_beta_column=3,
              count_column=1,
              data_cut=1000, # Selecting to 1000 expanded clonotypes
              type_of_data_cut='Num_Seq',
              aggregate_by_aa=True,  # Preventing redundancy bias of repeated clonotypes by aminoacid sequence
              classes=classes)

In [ ]:
folds = 100 # 100 fold Monte-Carlo cross validation
epochs_min = 25
size_of_net = 'small'
num_concepts=64
hinge_loss_t = 0.1
train_loss_min=0.1
seeds = np.array(range(folds))
graph_seed = 0

DTCR_CT12.Monte_Carlo_CrossVal(folds=folds,epochs_min=epochs_min,size_of_net=size_of_net,num_concepts=num_concepts,
                          train_loss_min=train_loss_min,combine_train_valid=True, # Enabling multi-sample dropout rate
                          hinge_loss_t=hinge_loss_t, # Enabling by class weigthing
                          multisample_dropout=True,
                          weight_by_class =True,
                          seeds=seeds,graph_seed=graph_seed, 
                          subsample=100 )  # Subsampling 100 sequences each fold during training

In [ ]:
DTCR_CT12.AUC_Curve(filename='AUC_ct1ct2.tif', title='AUC-ROC curves from Cluster 1 and cluster 2 Samples')

In [ ]:
DTCR_CT12.Motif_Identification('ct1', by_samples=True) # Enablig by sample motif identification instead of at sequence level
DTCR_CT12.Motif_Identification('ct2', by_samples=True)

In [10]:
# Data frame of each sequence's AUC for each clusters during model performance using the test partition

with open('ct12_model_preds.pkl','wb') as f:
    pickle.dump(DTCR_CT13.DFs_pred,f,protocol=4)

with open('ct12_model_seq_preds.pkl', 'wb') as f:
    pickle.dump((DTCR_CT13.predicted,DTCR_CT13.beta_sequences,DTCR_CT13.lb), f, protocol=4) 

with open('ct12_model_seq_preds.pkl', 'rb') as f:
    predicted, beta_sequences,lb = pickle.load(f)

df = pd.DataFrame()
df['beta_sequences'] = beta_sequences
df[lb.classes_[0]] = predicted[:,0]
df[lb.classes_[1]] = predicted[:,1]


In [11]:
sel = 0
cl = lb.classes_[sel]
df.sort_values(by=cl,inplace=True,ascending=False)

In [ ]:
fig12_ct1,ax = DTCR_CT23.Residue_Sensitivity_Logo(beta_sequences=np.array(df['beta_sequences'])[0:25] , # top 25 sequences with the highest AUC for cluster 1,
                              class_sel=cl,figsize=(5,10),background_color='white',Load_Prev_Data=False,
                              min_size=0.25,  low_color='blue', medium_color='yellow', high_color='red')
fig12_ct1.savefig('rsl_ct12_ct1.png',
            dpi=1200,facecolor='black')

In [ ]:
sel = 1
cl = lb.classes_[sel]
df.sort_values(by=cl,inplace=True,ascending=False)

In [ ]:
fig12_ct2,ax = DTCR_CT12.Residue_Sensitivity_Logo(beta_sequences=np.array(df['beta_sequences'])[0:25], # top 25 sequences with the highest AUC for cluster 2,
                              class_sel=cl,figsize=(5,10),background_color='white',Load_Prev_Data=False,
                              min_size=0.25,  low_color='blue', medium_color='yellow', high_color='red')
fig12_ct2.savefig('rsl_ct12_ct2.png',
            dpi=1200,facecolor='black')